In [64]:

api_access = "api_secret = "app_id = "app_key = "



In [60]:
#!/usr/bin/env python

import json
import requests
import base64
import uuid
import datetime
import hashlib
import hmac
import time
 
# Set up variables
APP_ID = "852f9bc3-2919-4001-905e-e75755d763a2"
APP_KEY = "c9f26e03-69be-4046-b286-4a9ccd60693d"
URI = "/api/ttp/url/decode-url"
EMAIL_ADDRESS = 'ACCESS_KEY = ''SECRET_KEY = ' 
# Supporting methods
def get_hdr_date():
    return datetime.datetime.utcnow().strftime("%a, %d %b %Y %H:%M:%S UTC")
 
 
def get_base_url(email_address):
    # Create post body for request
    post_body = dict()
    post_body['data'] = [{}]
    post_body['data'][0]['emailAddress'] = email_address
 
    # Create variables required for request headers
    request_id = str(uuid.uuid4())
    request_date = get_hdr_date()
    headers = {'x-mc-app-id': APP_ID, 'x-mc-req-id': request_id, 'x-mc-date': request_date}
 
    # Send request to API
    
    try:
        r = requests.post(url='https://api.mimecast.com/api/login/discover-authentication',
                          data=json.dumps(post_body), headers=headers)
        # Handle Rate Limiting
        if r.status_code == 429:
            time.sleep(r.headers['X-RateLimit-Reset'] * 1000)
    except Exception as e:
        quit()
 
    # Handle error from API
    if r.status_code != 200:
        quit()
 
    # Load response body as JSON
    resp_data = json.loads(r.text)

    # Look for api key in region region object to get base url
    if 'region' in resp_data["data"][0]:
        base_url = resp_data["data"][0]["region"]["api"].split('//')
        base_url = base_url[1]
    else:
        # Handle no region found, likely the email address was entered incorrectly
        quit()
 
    return base_url
 
 
def post_request(base_url, uri, post_body, access_key, secret_key):
 
    # Create variables required for request headers
    request_id = str(uuid.uuid4())
    request_date = get_hdr_date()
 
    unsigned_auth_header = '{date}:{req_id}:{uri}:{app_key}'.format(
        date=request_date,
        req_id=request_id,
        uri=uri,
        app_key=APP_KEY
    )
    hmac_sha1 = hmac.new(
        base64.b64decode(secret_key),
        unsigned_auth_header.encode(),
        digestmod=hashlib.sha1).digest()
    sig = base64.encodebytes(hmac_sha1).rstrip()
    headers = {
        'Authorization': 'MC ' + access_key + ':' + sig.decode(),
        'x-mc-app-id': APP_ID,
        'x-mc-date': request_date,
        'x-mc-req-id': request_id,
        'Content-Type': 'application/json'
    }
 
    try:
        # Send request to API
        r = requests.post(url='https://' + base_url + uri, data=json.dumps(post_body), headers=headers)
 
        # Handle Rate Limiting
        if r.status_code == 429:
            time.sleep(r.headers['X-RateLimit-Reset'] * 1000)
            r = requests.post(url='https://' + base_url + uri, data=json.dumps(post_body), headers=headers)
 
    # Handle errors
    except Exception as e:
        return 'error'
    # Handle errors from API
    if r.status_code != 200:
        return 'error'
 
    # Return response body and response headers
    return r.content, r.headers
 
 
def decode_mimecast_url(base_url,url, access_key, secret_key, now):
 
    
    uri = "/api/ttp/url/decode-url"
 
    # Build post body for request
    post_body = dict()
    post_body['data'] = [{}]
    post_body['data'][0]['url'] = url
 
    # Send request to API
    resp = post_request(base_url, uri, post_body, access_key, secret_key)
    return resp[0]
 

base_url = get_base_url(email_address=EMAIL_ADDRESS)
print(base_url)

  

za-api.mimecast.com


In [63]:
with open("links.txt","rb") as File:
    File = File.read().decode()
    links = File.splitlines()
    new_links = []

with open("converted_links.txt", "wb") as File:
    for link in links:
        res = decode_mimecast_url(base_url=base_url,url=link, access_key=ACCESS_KEY,secret_key=SECRET_KEY, now = time.time())
        res = res.decode()
        res = json.loads(res)
        new_link = res['data'][0]['url'] + "\n"
        new_link = new_link.encode()
        File.write(new_link)
  
        
    